In [1]:
import requests
import pandas as pd
import numpy as np
import nbconvert
import datetime
import warnings; warnings.simplefilter('ignore')
import plotly.graph_objects as go
from IPython.display import display, Markdown, Image, SVG
import re
import bmondata
from bmondata import Server
from dateutil.relativedelta import relativedelta
import urllib
import io
import requests
import math


In [2]:

def get_fuel_df(file_path):
    ak_energy_url = 'http://ak-energy-data.analysisnorth.com/'
    """Returns a Pandas DataFrame that is found at the 'file_path'
    below the Base URL for accessing data.  The 'file_path' should end
    with '.pkl' and points to a pickled, compressed (bz2), Pandas DataFrame.
    """
    b = requests.get(urllib.parse.urljoin(ak_energy_url, file_path)).content
    df = pd.read_pickle(io.BytesIO(b), compression='bz2')
    return df

In [3]:
heat_flow_sensor_dict = {
    'Quinhagak': 'quinhagak_hr_heat_rate'
}

total_heat_sensor_dict = {
    'Quinhagak': 'quinhagak_hr_total_heat'
}

fuel_price_Dict = {
    'Quinhagak': 6.13
}

ak_warm_city_building_dict= {
    'Quinhagak': 'Quinhagak'
}

In [4]:
# Papermill variables to change to look at different buildings / organizations 
building_id = 32
server_web_address = 'http://rm.anthc.webfactional.com'



In [5]:
averaging_time_length = 24 #average data over 1 day timespan
kbtu_per_gallon_oil = 134
boiler_efficiency = .75 

In [6]:
server = Server(server_web_address)
building_df = server.buildings(building_id)
current_building_name = building_df[0]['title']
sensors=building_df[0]['sensors']

heat_flow_sensor = []
total_heat_sensor = []
for sensor in sensors:
    if sensor['sensor_id'] == heat_flow_sensor_dict[current_building_name]:
        heat_flow_sensor = sensor
        
    if sensor['sensor_id'] == total_heat_sensor_dict[current_building_name]:
        total_heat_sensor = sensor 


In [7]:
all_buildings = server.buildings()
org_df = pd.DataFrame(all_buildings)

In [8]:
title_md = '''# Heat Flow for {} building'''
title_md = title_md.format(current_building_name)

In [9]:
Markdown(title_md)

# Heat Flow for Quinhagak building

In [10]:
averaging_string = str(averaging_time_length) + 'h'
fifteen_min_averages = server.sensor_readings((heat_flow_sensor_dict[current_building_name], 'heat_flow'),
                                          start_ts = datetime.datetime.now() - relativedelta(years=1),
                                            end_ts = datetime.datetime.now(),
                                          averaging = averaging_string)

In [11]:
# Get rid of any potential erroneous data that is listed as a negative electric usage
fifteen_min_averages = fifteen_min_averages.query("heat_flow >= 0")


In [12]:
fifteen_min_averages = fifteen_min_averages.reset_index()

In [13]:
fifteen_min_averages = fifteen_min_averages.rename(columns={'index':'datetime_col'})

In [14]:
fifteen_min_averages['dayofweek'] = fifteen_min_averages.datetime_col.apply(lambda x: x.dayofweek)

In [15]:
fifteen_min_averages['date'] = fifteen_min_averages.datetime_col.apply(lambda x: x.date())

In [16]:
fifteen_min_averages['time'] = fifteen_min_averages.datetime_col.apply(lambda x: x.time())

In [17]:
current_week_start = fifteen_min_averages.date.iloc[-1] - pd.offsets.Day(7)

In [18]:
# Create dataframes for each of the last three weeks for graphing comparison purposes.
last_week_start = current_week_start - pd.offsets.Day(7)
last_week_df = fifteen_min_averages.query("datetime_col >= @last_week_start")
last_week_df = last_week_df.query("datetime_col < @current_week_start")

two_weeks_ago_start = last_week_start - pd.offsets.Day(7)
two_weeks_ago_df = fifteen_min_averages.query("datetime_col >= @two_weeks_ago_start")
two_weeks_ago_df = two_weeks_ago_df.query("datetime_col < @last_week_start")

three_weeks_ago_start = two_weeks_ago_start - pd.offsets.Day(7)
three_weeks_ago_df = fifteen_min_averages.query("datetime_col >= @three_weeks_ago_start")
three_weeks_ago_df = three_weeks_ago_df.query("datetime_col < @two_weeks_ago_start")

In [19]:
# Adjust the datetime column so the visualization will have the heat flow data overlaying the current week.
last_week_df['adjusted_datetime_col'] = last_week_df.datetime_col.apply(lambda x: x + pd.offsets.Day(7))
two_weeks_ago_df['adjusted_datetime_col'] = two_weeks_ago_df.datetime_col.apply(lambda x: x + pd.offsets.Day(14))
three_weeks_ago_df['adjusted_datetime_col'] = three_weeks_ago_df.datetime_col.apply(lambda x: x + pd.offsets.Day(21))

In [20]:
current_week = fifteen_min_averages.query("datetime_col >= @current_week_start")

In [21]:
historical_complete = fifteen_min_averages.query("datetime_col < @current_week_start")

In [22]:
day_avgs = historical_complete.groupby(['dayofweek', 'time']).mean()
day_avgs = day_avgs.reset_index()

In [23]:
current_week_w_day_avgs = pd.merge(current_week, day_avgs, how='left',
                                  left_on=['dayofweek', 'time'],
                                  right_on=['dayofweek', 'time'], 
                                  suffixes=('', '_historical_avg'))

In [24]:
current_week_w_day_avgs['current_use_vs_historical_difference'] = current_week_w_day_avgs.heat_flow - current_week_w_day_avgs.heat_flow

In [25]:
# Define anomalous range as three standard deviations above the mean
three_std = current_week_w_day_avgs.current_use_vs_historical_difference.mean() + current_week_w_day_avgs.current_use_vs_historical_difference.std() * 3

In [26]:
# This works, but the graph looks odd unless we get one data point on either side to connect them
current_week_w_day_avgs['heat_flow_anomalies'] = np.where(current_week_w_day_avgs.current_use_vs_historical_difference > three_std,
                                                            current_week_w_day_avgs.heat_flow,
                                                            np.nan)

In [27]:
anomaly_df = current_week_w_day_avgs.query("heat_flow_anomalies == heat_flow_anomalies")

In [28]:
diff_series = np.diff(anomaly_df.index, n=1)

In [29]:
diff_series = np.insert(diff_series, 0, 0)
anomaly_df['diff_series'] = diff_series

In [30]:
# This keeps track of the number of groups of distinct anomalous periods in the data
group_counter = 0

# This padding is a cut-off used to group together time series indices that are close 
# enough to still be considered a group (e.g. there might be an anomaly at one time,
# followed by another anomalous reading 45 minutes later, which should probably all just
# be considered the same group)
index_padding = 4

for idx, row in anomaly_df.iterrows():
    if row['diff_series'] <= index_padding:
        anomaly_df.at[idx, 'group_number'] = group_counter
    else:
        group_counter += 1
        anomaly_df.at[idx, 'group_number'] = group_counter

In [31]:
anomaly_max = anomaly_df.groupby(['group_number']).max()[['datetime_col', 'heat_flow']]
anomaly_min = anomaly_df.groupby(['group_number']).min()[['datetime_col', 'heat_flow']]

In [32]:
anomaly_groups = pd.merge(anomaly_max, anomaly_min, how='outer',
                         left_index=True, right_index=True, 
                         suffixes=('_max', '_min'))

In [34]:
if anomaly_groups.datetime_col_min.isna().all():
    anomaly_rectangles = []
    image_list = []
else:
    anomaly_rectangles = []
    image_list = []
    for idx, row in anomaly_groups.iterrows():
        
        anomaly_start = row['datetime_col_min']
        anomaly_end = row['datetime_col_max']
        anomaly_midpoint = ((anomaly_end - anomaly_start) / 2) + anomaly_start
        
        anomaly_rectangles.append({'type':'rect',
                            'xref':'x',
                            'yref':'paper',
                            'x0':anomaly_start,
                            'y0':0,
                            'x1':anomaly_end,
                            'y1':1,
                            'fillcolor':('rgb(205, 12, 24)'),
                            'opacity':0.5,
                            'line': {
                                'width':1,
                            }
                            })
        
        image_list.append({'source': 'https://github.com/dustin-cchrc/cchrc_python_for_non_programmers/blob/master/energy_savings_icon.png',
                'xref': "paper",
                'yref': "paper",
                'x': 1.0, #anomaly_midpoint ,
                'y': 0.9,
                'sizex': 0.2,
                'sizey': 0.2,
                'xanchor': "right",
                'yanchor': "bottom"})

In [39]:
this_week = go.Scatter(x = current_week_w_day_avgs.datetime_col,
                   y = averaging_time_length * current_week_w_day_avgs.heat_flow,
                   line = dict(color = '#ca0020'),
                   name='Current week heat flow')

last_week = go.Scatter(x = last_week_df.adjusted_datetime_col,
                      y = averaging_time_length * last_week_df.heat_flow,
                       line = dict(dash = 'dashdot',
                                   color = '#3182bd'),
                       name = "Heat flow last week")

two_weeks_ago = go.Scatter(x = two_weeks_ago_df.adjusted_datetime_col,
                          y = averaging_time_length * two_weeks_ago_df.heat_flow,
                           line = dict(dash = 'dashdot',
                                       color = '#6baed6'),
                           name = "Heat flow 2 weeks ago")

three_weeks_ago = go.Scatter(x = three_weeks_ago_df.adjusted_datetime_col,
                            y = averaging_time_length * three_weeks_ago_df.heat_flow,
                             line = dict(dash = 'dashdot',
                                         color = '#bdd7e7'),
                            name = "Heat flow 3 weeks ago")
    
layout = dict(title = 'Heat Flow: Current vs. Recent Weeks',
              xaxis = dict(title='Date and Time'),
              yaxis = dict(title='Heat Flow (KBTU, ' + str(averaging_time_length) + ' hour usage)')
             )

data = [this_week, last_week, two_weeks_ago, three_weeks_ago]

fig = go.Figure(dict(data=data, layout=layout))

fig.show()

In [36]:
fuel_per_kbtu= 1/(kbtu_per_gallon_oil*boiler_efficiency)

this_week = go.Scatter(x = current_week_w_day_avgs.datetime_col,
                   y = fuel_per_kbtu*averaging_time_length * current_week_w_day_avgs.heat_flow,
                   line = dict(color = '#ca0020'),
                   name='Current week savings')

last_week = go.Scatter(x = last_week_df.adjusted_datetime_col,
                      y = fuel_per_kbtu*averaging_time_length * last_week_df.heat_flow,
                       line = dict(dash = 'dashdot',
                                   color = '#3182bd'),
                       name = "Savings last week")

two_weeks_ago = go.Scatter(x = two_weeks_ago_df.adjusted_datetime_col,
                          y = fuel_per_kbtu*averaging_time_length * two_weeks_ago_df.heat_flow,
                           line = dict(dash = 'dashdot',
                                       color = '#6baed6'),
                           name = "Savings 2 weeks ago")

three_weeks_ago = go.Scatter(x = three_weeks_ago_df.adjusted_datetime_col,
                            y = fuel_per_kbtu*averaging_time_length * three_weeks_ago_df.heat_flow,
                             line = dict(dash = 'dashdot',
                                         color = '#bdd7e7'),
                            name = "Savings 3 weeks ago")
    
layout = dict(title = 'Heat Recovery Fuel Saved',
              xaxis = dict(title='Date and Time'),
              yaxis = dict(title='Heat Recovery Fuel Savings (Gallons, ' + str() + ' hour savings)')
             )

data = [this_week, last_week, two_weeks_ago, three_weeks_ago]

fig = go.Figure(dict(data=data, layout=layout))

fig.show()

In [37]:
fuel_price_df = get_fuel_df('city-util/proc/city.pkl')
fuel_price_df = fuel_price_df.loc[fuel_price_df['Name'] == current_building_name]
fuel_price=fuel_price_df.iloc[0]['GasPrice']
if (math.isnan(fuel_price)):
    if current_building_name in fuel_price_Dict:
        fuel_price = fuel_price_Dict[current_building_name]
if (not math.isnan(fuel_price)):
    cost_per_kbtu= fuel_price/(kbtu_per_gallon_oil*boiler_efficiency)
    this_week = go.Scatter(x = current_week_w_day_avgs.datetime_col,
                       y = cost_per_kbtu*averaging_time_length * current_week_w_day_avgs.heat_flow,
                       line = dict(color = '#ca0020'),
                       name='Current week savings')

    last_week = go.Scatter(x = last_week_df.adjusted_datetime_col,
                          y = cost_per_kbtu*averaging_time_length * last_week_df.heat_flow,
                           line = dict(dash = 'dashdot',
                                       color = '#3182bd'),
                           name = "Savings last week")

    two_weeks_ago = go.Scatter(x = two_weeks_ago_df.adjusted_datetime_col,
                              y = cost_per_kbtu*averaging_time_length * two_weeks_ago_df.heat_flow,
                               line = dict(dash = 'dashdot',
                                           color = '#6baed6'),
                               name = "Savings 2 weeks ago")

    three_weeks_ago = go.Scatter(x = three_weeks_ago_df.adjusted_datetime_col,
                                y = cost_per_kbtu*averaging_time_length * three_weeks_ago_df.heat_flow,
                                 line = dict(dash = 'dashdot',
                                             color = '#bdd7e7'),
                                name = "Savings 3 weeks ago")

    layout = dict(title = 'Heat Recovery Money Saved',
                  xaxis = dict(title='Date and Time'),
                  yaxis = dict(title='Heat Recovery Cost Savings (Dollars, ' + str() + ' hour savings)')
                 )

    data = [this_week, last_week, two_weeks_ago, three_weeks_ago]

    fig = go.Figure(dict(data=data, layout=layout))

    fig.show()
